In [1]:
%cd ..

/workspaces/Code


In [2]:
%matplotlib inline

from __future__ import annotations
import os

import tensorflow as tf
from ModelBuilders import TestVgg16NetBuilder, TestMobileNetBuilder, TestResNetBuilder, ModelOptions, BaseModelBuilder

from map_provider import ImageProvider, ImageProjection, MapProvider
from ExperimentRunners.OffsetRunner import OffsetRunnerOptions, OffsetRunner, OffsetDataSaver

from ExperimentRunners.OffsetRunner.Models import ReferencePoint
from vector import Vector2D

from color_generator import ColorGenerator

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [47]:
thresh = 0.7043
model_builder = TestVgg16NetBuilder()

# thresh = 0.6914
# model_builder = TestMobileNetBuilder()

# thresh = 0.7352
# options = ModelOptions(
#     builder_label="Experimental/NewTraining/ResNet50",
#     model_nn=50,
#     ingest_dense=128,
#     output_dense=8,
#     trainable_from_index=0,
#     epochs=6,
#     batch_size=16,
# )

# model_builder = TestResNetBuilder(options)

In [4]:
def _experiment_folder(model_builder: BaseModelBuilder):
    options = model_builder.get_options()
    label = options.builder_label
    representation = options.representation()

    experiment_folder = f"Data/ExampleOffset/{label}/{representation}"
    os.makedirs(experiment_folder, exist_ok=True)

    return experiment_folder

In [5]:
def generate_reference_points():
    reference_points = [
        ReferencePoint(Vector2D(5881, 1656), 'miškai'),
        ReferencePoint(Vector2D(15650, 24030), 'miškai'),
        ReferencePoint(Vector2D(8650, 2000), 'miškai'),
        ReferencePoint(Vector2D(5846, 4752), 'daugiabučiai'),
        ReferencePoint(Vector2D(8046, 19828), 'daugiabučiai'),
        ReferencePoint(Vector2D(3330, 9590), 'daugiabučiai'),
        ReferencePoint(Vector2D(5310, 7960), 'daugiabučiai'),
        ReferencePoint(Vector2D(2501, 3402), 'daugiabučiai'),
        ReferencePoint(Vector2D(1973, 22699), 'pievos'),
        ReferencePoint(Vector2D(4780, 4160), 'keliai'),
        ReferencePoint(Vector2D(14130, 1500), 'keliai'),
        ReferencePoint(Vector2D(24230, 10280), 'vandens telkiniai'),
        ReferencePoint(Vector2D(9960, 16340), 'individualūs namai'),
        ReferencePoint(Vector2D(7781, 5645), 'miškai, keliai'),
        ReferencePoint(Vector2D(25000, 24570), 'miškai, keliai'),
        ReferencePoint(Vector2D(18543, 9141), 'miškai, daugiabučiai'),
        ReferencePoint(Vector2D(9250, 20580), 'daugiabučiai, keliai'),
        ReferencePoint(Vector2D(25678, 13083), 'daugiabučiai, individualūs namai'),
        ReferencePoint(Vector2D(3298, 25260), 'keliai, pievos'),
        ReferencePoint(Vector2D(2013, 3788), 'keliai, individualūs namai'),
    ]

    return reference_points

In [7]:
city_image = ImageProvider(image_path="City/NewCut/City_2017.jpg")
drone_image = ImageProvider(image_path="City/NewCut/City_2016.jpg")

projection = ImageProjection(
    position=Vector2D(8000, 8000),
    size=Vector2D(6000, 6000))

city_provider = MapProvider(
    image_provider=city_image,
    crop_size=672,
    projection=None)

drone_provider = MapProvider(
    image_provider=drone_image,
    crop_size=672,
    projection=None)

Loading image: City/NewCut/City_2017.jpg
Loading image: City/NewCut/City_2016.jpg


In [48]:
options = OffsetRunnerOptions(
    model_builder=model_builder,
    city_map=city_provider,
    drone_map=drone_provider,
    reference_points=generate_reference_points())

offset_runner = OffsetRunner(options)
stats = offset_runner.run()

Running offset runner tests


Running point offsets: 100%|██████████| 20/20 [00:56<00:00,  2.83s/it]

In [49]:
label_colors = {
    'daugiabučiai': '#8e44ad',
    'keliai': '#34495e',
    'individualūs namai': '#c0392b',
    'pievos': '#f1c40f',
    'miškai': '#16a085',
    'vandens telkiniai': '#3498db',
}

color_generator = ColorGenerator("#2c3e50", label_colors)

offset_saver = OffsetDataSaver(_experiment_folder(model_builder), offset_runner.target_distances, color_generator, thresh)
offset_saver.save(stats)

Saving offset data
